# Sistema Interconectado Nacional de Energía Eléctrica de Colombia (SIN)
EL Sistema Interconectado Nacional es el sistema compuesto por los siguientes elementos conectados entre sí: las plantas y equipos de generación, la red de interconexión, las redes regionales e interregionales de transmisión, las redes de distribución, y las cargas eléctricas de los usuarios. [XM](https://www.xm.com.co/corporativo/Paginas/Herramientas/glosario.aspx)

En el presente caso de estudio se pretende mostrar las principales variables que interactuan en el SIN e inciden los mercado de Energía Eléctrica y Gas Natural




In [7]:
from bs4 import BeautifulSoup as bfs4
import pandas as pd 
import numpy as np 
import matplotlib as plt
import requests as rq
from datetime import date
from datetime import datetime

http://portalbissrs.xm.com.co/oferta/Histricos/Generacion_(kWh)_TRIM1_2020.xlsx

In [ ]:

#Estructura de los nombres de los archivos a descargar PORTALBI-XM:
now = datetime.now()
year = now.strftime('%Y')
month = now.strftime('%m')
day = now.strftime('%d')
day = int(day)
dayres3 = "0" + str(day - 3) #rezago de 3 días
g = "_"

m = int(month)
if m >= 1 and m <=3:
    trim = "1"
elif m >= 4 and m <=6:
    trim = "2"
elif m >= 7 and m <=9:
    trim = "3"
else:
    trim = "4"

names = []
namegen = year + g + month + g + dayres3 + g + "Generacion_(kWh)" + g + year + "TRIM" + trim + ".xlsx"
nameintecu = year + g + month + g + dayres3 + g + "Exportacion_Importacion_Ecuador" + g + year + ".xlsx"
nameintven = year + g + month + g + dayres3 + g + "Exportacion_Importacion_Venezuela" + g + year + ".xlsx"
namecomb = year + g + month + g + dayres3 + g + "Consumo_De_Combustible_(MBTU)" + g + year + ".xlsx"
names.append(namegen)
names.append(nameintecu)
names.append(nameintven)
names.append(namecomb)

import requests
links = []
linkgen = "http://portalbissrs.xm.com.co/oferta/Histricos/Generacion_(kWh)_TRIM" + trim + g + year + ".xlsx"
linkintecu = "http://portalbissrs.xm.com.co/dmnd/Histricos/Exportacion_Importacion_Ecuador" + g + year + ".xlsx"
linkintven = "http://portalbissrs.xm.com.co/dmnd/Histricos/Exportacion_Importacion_Venezuela" + g + year + ".xlsx"
linkcomb = "http://portalbissrs.xm.com.co/oferta/Histricos/Consumo_De_Combustible_(MBTU)" + g + year + ".xlsx"
links.append(linkgen)
links.append(linkintecu)
links.append(linkintven)
links.append(linkcomb)

resp = requests.get(linkgen)
output = open(namegen, 'wb')
output.write(resp.content)
output.close()
print("Archivo descargado:",namegen)

resp = requests.get(linkintecu)
output = open(nameintecu, 'wb')
output.write(resp.content)
output.close()
print("Archivo descargado:",nameintecu)

resp = requests.get(linkintven)
output = open(nameintven, 'wb')
output.write(resp.content)
output.close()
print("Archivo descargado:",nameintven)

resp = requests.get(linkcomb)
output = open(namecomb, 'wb')
output.write(resp.content)
output.close()
print("Archivo descargado:",namecomb)